In [10]:
#importations
from matplotlib import pyplot as plt
import numpy as np
import cv2
import glob

In [7]:
# model functions

def relu(x):
    return np.maximum(0,x)
    
def sigmoid(x):
    return 1/(1+np.exp(-x))

def relu_derivative(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

def sigmoid_derivative(x):
    return np.multiply(x,1-x)
    
def initialize_variables(n, NUMBER_OF_LAYERS, LAYERS_DIM):
    
    parameters = {}
    
    for i in range(1, NUMBER_OF_LAYERS):
        W = np.random.randn(LAYERS_DIM[i],LAYERS_DIM[i-1]) * 0.01
        b = np.zeros((LAYERS_DIM[i],1))
        parameters["W"+str(i)] = W
        parameters["b"+str(i)] = b
        
    return parameters

def forward_one_layer(W, b, X, activation = "relu"):
    
    Z = np.dot(W,X) + b
    
    if activation == "relu":
        A = relu(Z)
    elif activation == "sigmoid":
        A = sigmoid(Z)
        
    cache = (W,b,X,Z,A)
        
    return cache
    
def forward_all_layers(X, parameters):
    
    caches = []
    NUMBER_OF_LAYERS = (len(parameters) // 2) + 1
    #print(NUMBER_OF_LAYERS)
    caches.append((None,None,None,None,X))
    
    for i in range(1, NUMBER_OF_LAYERS-1):
        cache = forward_one_layer(parameters["W"+str(i)], parameters["b"+str(i)], caches[i-1][4], "relu")
        caches.append(cache)
        
    cache = forward_one_layer(parameters["W"+str(NUMBER_OF_LAYERS-1)], parameters["b"+str(NUMBER_OF_LAYERS-1)], caches[NUMBER_OF_LAYERS-2][4], "sigmoid")
    caches.append(cache)
    
    return caches
    
        
def calculate_cost(A, Y):
    
    m = A.shape[1]
    
    cost = (-1/m)*(np.sum(np.multiply(Y,np.log(A)) + np.multiply(1-Y,np.log(1-A))))
    return cost

def backward_one_layer(dA, cache, activation = "relu"):
    
    W = cache[0]
    A_prev = cache[2]
    A = cache[4]
    m = A.shape[1]
    
    if activation == "relu": 
        dZ = np.multiply(dA, relu_derivative(A))
    elif activation == "sigmoid":
        dZ = np.multiply(dA, sigmoid_derivative(A))
        
    dW = (1/m)*np.dot(dZ,A_prev.T)
    db = (1/m)*np.sum(dZ,axis=1,keepdims=True)
    dA_prev = np.dot(W.T,dZ)
    
    dcache = (dW, db, dZ, dA_prev)
    
    return dcache
    
def backward_all_layers(Y, caches):
    
    dcaches = []
    dcaches.append((None, None, None, None))
    NUMBER_OF_LAYERS = len(caches)
    A = caches[NUMBER_OF_LAYERS-1][4]
    
    dA = np.divide(1-Y,1-A) - np.divide(Y,A)
    dcache = backward_one_layer(dA, caches[NUMBER_OF_LAYERS-1], activation = "sigmoid")
    dcaches.append(dcache)
    
    for i in range(1, NUMBER_OF_LAYERS-1):
        dA = dcaches[i][3]
        dcache = backward_one_layer(dA, caches[NUMBER_OF_LAYERS - 1 - i], activation = "relu")
        dcaches.append(dcache)
    
    return dcaches
    
def update_parameters(parameters, dcaches, learning_rate = 0.001):
    
    NUMBER_OF_LAYERS = (len(parameters) // 2) + 1
    
    for i in range(1, NUMBER_OF_LAYERS):
        parameters["W"+str(i)] = parameters["W"+str(i)] - learning_rate * dcaches[NUMBER_OF_LAYERS-i][0]
        parameters["b"+str(i)] = parameters["b"+str(i)] - learning_rate * dcaches[NUMBER_OF_LAYERS-i][1]
    
    return parameters

def model(X, Y, m, NUMBER_OF_LAYERS, LAYERS_DIM, num_iterations = 2, learning_rate = 0.001):
    
    parameters = initialize_variables(X.shape[0], NUMBER_OF_LAYERS, LAYERS_DIM)
    costs = []
    iterations = []
    
    for i in range(num_iterations):
        caches = forward_all_layers(X, parameters)
        cost = calculate_cost(caches[NUMBER_OF_LAYERS-1][4], Y)
        if i%1 == 0:
            print("Iteration number : "+ str(i) +"cost is : " + str(cost))
            costs.append(cost)
            iterations.append(i)
        dcaches = backward_all_layers(Y, caches)
        parameters = update_parameters(parameters, dcaches, learning_rate = 0.001)
        
    plt.plot(iterations, costs)
    plt.show()
    return parameters
    
def predict(X, parameters):
    caches = forward_all_layers(X, parameters)
    Y = caches[len(caches)-1][4]
    Y[Y<=0.5] = 0
    Y[Y>0.5] = 1
    return Y
    
    

In [15]:
pathA = "sample/A/*"
pathB = "sample/B/*"
images_orig_A = np.array([cv2.imread(file) for file in glob.glob(pathA)])
images_orig_B = np.array([cv2.imread(file) for file in glob.glob(pathB)])
images_orig = np.concatenate((images_orig_A, images_orig_B))
images_flatten = images_orig/255
print(images_flatten.shape)
images = images_flatten.reshape(images_flatten.shape[0],images_flatten.shape[1]*images_flatten.shape[2]*images_flatten.shape[3]).T
print(images.shape)
Y_A = np.ones((1,images_orig_A.shape[0]))
Y_B = np.zeros((1,images_orig_B.shape[0]))
Y = np.concatenate((Y_A, Y_B), axis=1)
print(Y.shape)

(21018, 45, 45, 3)
(6075, 21018)
(1, 21018)


In [ ]:
parameters = None
parameters = model(images, Y, images.shape[1], 4, [images.shape[0], 5, 2, 1], 20000, 1.2)

cost is : 0.6931465302705393
cost is : 0.693138716555131
cost is : 0.6931309067463601
cost is : 0.6931231008423021
cost is : 0.6931152988409134
cost is : 0.6931075007402427
cost is : 0.6930997065383399
cost is : 0.6930919162332554
cost is : 0.6930841298230411
cost is : 0.6930763473057495
cost is : 0.6930685686794343
cost is : 0.6930607939421499
cost is : 0.6930530230919517
cost is : 0.6930452561269359
cost is : 0.6930374930456914
cost is : 0.6930297338457048
cost is : 0.6930219785250352
cost is : 0.6930142270817433
cost is : 0.6930064795138903
cost is : 0.692998735819538
cost is : 0.6929909959967496
cost is : 0.6929832600435895
cost is : 0.6929755279581226
cost is : 0.6929677997384088
cost is : 0.6929600753823818
cost is : 0.6929523548882487
cost is : 0.6929446382540785
cost is : 0.692936925477941
cost is : 0.6929292165579068
cost is : 0.692921511492048
cost is : 0.6929138102784368
cost is : 0.6929061129151469
cost is : 0.6928984194002529
cost is : 0.6928907297318303
cost is : 0.692883

cost is : 0.6910772958128307
cost is : 0.6910705165937487
cost is : 0.6910637407636352
cost is : 0.6910569683207929
cost is : 0.6910501992634211
cost is : 0.6910434335898241
cost is : 0.6910366712983068
cost is : 0.6910299123871754
cost is : 0.6910231568547364
cost is : 0.6910164046993443
cost is : 0.6910096559193107
cost is : 0.6910029105129066
cost is : 0.6909961684785514
cost is : 0.6909894298144696
cost is : 0.6909826945190214
cost is : 0.6909759625904361
cost is : 0.6909692340270275
cost is : 0.6909625088271094
cost is : 0.6909557869889973
cost is : 0.690949068511007
cost is : 0.6909423533914554
cost is : 0.6909356416286602
cost is : 0.6909289332209396
cost is : 0.6909222281666132
cost is : 0.6909155264640655
cost is : 0.690908828111622
cost is : 0.6909021331075355
cost is : 0.6908954414501284
cost is : 0.6908887531377246
cost is : 0.6908820681686478
cost is : 0.6908753865412235
cost is : 0.6908687082537665
cost is : 0.6908620333044521
cost is : 0.6908553616917704
cost is : 0.6908

cost is : 0.6892819880814323
cost is : 0.689276106162628
cost is : 0.6892702271829585
cost is : 0.68926435114095
cost is : 0.689258478035129
cost is : 0.6892526078640233
cost is : 0.6892467406261611
cost is : 0.6892408763200715
cost is : 0.689235014944284
cost is : 0.6892291564973293
cost is : 0.6892233009777385
cost is : 0.6892174483840435
cost is : 0.6892115987147772
cost is : 0.6892057519684728
cost is : 0.6891999081436645
cost is : 0.6891940672388871
cost is : 0.6891882292526763
cost is : 0.689182394183595
cost is : 0.689176562030227
cost is : 0.6891707327910365
cost is : 0.6891649064645623
cost is : 0.6891590830493699
cost is : 0.6891532625440524
cost is : 0.6891474449470707
cost is : 0.6891416302569667
cost is : 0.689135818472282
cost is : 0.6891300095915597
cost is : 0.6891242036133434
cost is : 0.6891184005361775
cost is : 0.6891126003586068
cost is : 0.6891068030791807
cost is : 0.6891010086965527
cost is : 0.6890952172091596
cost is : 0.6890894286155491
cost is : 0.6890836429

cost is : 0.6877242574807647
cost is : 0.6877191537294539
cost is : 0.6877140525269
cost is : 0.6877089538718244
cost is : 0.6877038577629485
cost is : 0.6876987641989944
cost is : 0.6876936731786855
cost is : 0.687688584700745
cost is : 0.6876834987639229
cost is : 0.6876784153669839
cost is : 0.6876733345085881
cost is : 0.6876682561874612
cost is : 0.6876631804023301
cost is : 0.6876581071519379
cost is : 0.6876530364350636
cost is : 0.6876479682503694
cost is : 0.6876429025965842
cost is : 0.6876378394724384
cost is : 0.6876327788766622
cost is : 0.6876277208079868
cost is : 0.6876226652651444
cost is : 0.6876176122468769
cost is : 0.6876125617520317
cost is : 0.6876075137793227
cost is : 0.6876024683273801
cost is : 0.6875974253949386
cost is : 0.6875923849807787
cost is : 0.6875873470836444
cost is : 0.6875823117022843
cost is : 0.6875772788353716
cost is : 0.6875722484816447
cost is : 0.6875672206398423
cost is : 0.6875621953087039
cost is : 0.6875571724869697
cost is : 0.687552

cost is : 0.6863725505294878
cost is : 0.6863681215646624
cost is : 0.6863636948099346
cost is : 0.6863592702641947
cost is : 0.6863548479263343
cost is : 0.6863504277952451
cost is : 0.6863460098698194
cost is : 0.6863415941489502
cost is : 0.6863371806315313
cost is : 0.6863327693164563
cost is : 0.68632836020262
cost is : 0.6863239532889177
cost is : 0.6863195485742452
cost is : 0.686315146057534
cost is : 0.6863107457376847
cost is : 0.6863063476135556
cost is : 0.6863019516840444
cost is : 0.6862975579480496
cost is : 0.6862931664044704
cost is : 0.6862887770522061
cost is : 0.6862843898901574
cost is : 0.6862800049172244
cost is : 0.6862756221323087
cost is : 0.6862712415343284
cost is : 0.6862668631222245
cost is : 0.6862624868948598
cost is : 0.6862581128511839
cost is : 0.6862537409900396
cost is : 0.6862493713103313
cost is : 0.6862450038109643
cost is : 0.6862406384908443
cost is : 0.6862362753488775
cost is : 0.6862319143839705
cost is : 0.6862275555950306
cost is : 0.68622

...

cost is : 0.6782668977351908
cost is : 0.6782664914859496
cost is : 0.6782660854360292
cost is : 0.678265679585331
cost is : 0.678265273933757
cost is : 0.678264868481209
cost is : 0.6782644632275885
cost is : 0.6782640581727974
cost is : 0.6782636533167379
cost is : 0.6782632486593115
cost is : 0.6782628442004208
cost is : 0.6782624399399672
cost is : 0.6782620358778529
cost is : 0.6782616320139805
cost is : 0.6782612283482516
cost is : 0.6782608248805689
cost is : 0.6782604216108344
cost is : 0.6782600185389508
cost is : 0.6782596156648201
cost is : 0.6782592129883447
cost is : 0.6782588105094274
cost is : 0.6782584082279703
cost is : 0.6782580061438765
cost is : 0.6782576042570484
cost is : 0.6782572025673885
cost is : 0.6782568010747996
cost is : 0.6782563997791846
cost is : 0.6782559986804463
cost is : 0.6782555977784874
cost is : 0.678255197073211
cost is : 0.67825479656452
cost is : 0.6782543962523173
cost is : 0.6782539961365062
cost is : 0.6782535962169894
cost is : 0.67825319

cost is : 0.678159174005016
cost is : 0.6781588205892606
cost is : 0.6781584673467703
cost is : 0.6781581142774599
cost is : 0.6781577613812437
cost is : 0.6781574086580368
cost is : 0.6781570561077539
cost is : 0.6781567037303093
cost is : 0.6781563515256183
cost is : 0.6781559994935954
cost is : 0.6781556476341557
cost is : 0.6781552959472141
cost is : 0.6781549444326856
cost is : 0.6781545930904852
cost is : 0.678154241920528
cost is : 0.6781538909227289
cost is : 0.6781535400970032
cost is : 0.6781531894432661
cost is : 0.6781528389614327
cost is : 0.6781524886514185
cost is : 0.6781521385131388
cost is : 0.6781517885465087
cost is : 0.6781514387514438
cost is : 0.6781510891278594
cost is : 0.67815073967566
cost is : 0.678150390394758
cost is : 0.6781500412850832
cost is : 0.6781496923465509
cost is : 0.6781493435790771
cost is : 0.6781489949825775
cost is : 0.6781486465569677
cost is : 0.6781482983021634
cost is : 0.6781479502180805
cost is : 0.678147602304635
cost is : 0.67814725

cost is : 0.6780654550760583
cost is : 0.678065147590234
cost is : 0.6780648402550465
cost is : 0.6780645330704215
cost is : 0.678064226036285
cost is : 0.6780639191525628
cost is : 0.6780636124191809
cost is : 0.6780633058360652
cost is : 0.6780629994031304
cost is : 0.6780626931202905
cost is : 0.6780623869874949
cost is : 0.6780620810046697
cost is : 0.6780617751717412
cost is : 0.6780614694886354
cost is : 0.6780611639552788
cost is : 0.6780608585715973
cost is : 0.6780605533375175
cost is : 0.6780602482529654
cost is : 0.6780599433178678
cost is : 0.6780596385321508
cost is : 0.6780593338957409
cost is : 0.6780590294085647
cost is : 0.6780587250705484
cost is : 0.678058420881619
cost is : 0.6780581168417029
cost is : 0.6780578129507266
cost is : 0.678057509208617
cost is : 0.6780572056153094
cost is : 0.6780569021707357
cost is : 0.6780565988748085
cost is : 0.6780562957274553
cost is : 0.6780559927286023
cost is : 0.6780556898781784
cost is : 0.6780553871761313
cost is : 0.678055

cost is : 0.6779839118738938
cost is : 0.6779836443222849
cost is : 0.6779833769016573
cost is : 0.6779831096119465
cost is : 0.6779828424530884
cost is : 0.677982575425018
cost is : 0.6779823085276716
cost is : 0.6779820417609845
cost is : 0.6779817751248927
cost is : 0.6779815086193316
cost is : 0.6779812422442373
cost is : 0.6779809759995453
cost is : 0.677980709885192
cost is : 0.6779804439011127
cost is : 0.6779801780472434
cost is : 0.6779799123235204
cost is : 0.6779796467298794
cost is : 0.6779793812662565
cost is : 0.6779791159325878
cost is : 0.677978850728809
cost is : 0.6779785856548401
cost is : 0.677978320710604
cost is : 0.6779780558960665
cost is : 0.6779777912111637
cost is : 0.677977526655832
cost is : 0.6779772622300073
cost is : 0.6779769979336264
cost is : 0.6779767337666253
cost is : 0.6779764697289643
cost is : 0.6779762058206059
cost is : 0.6779759420414366
cost is : 0.6779756783913927
cost is : 0.6779754148704109
cost is : 0.6779751514784275
cost is : 0.6779748

cost is : 0.6779129557190903
cost is : 0.6779127228935236
cost is : 0.6779124901818623
cost is : 0.6779122575840504
cost is : 0.6779120251000319
cost is : 0.6779117927297509
cost is : 0.6779115604731514
cost is : 0.6779113283301776
cost is : 0.677911096300774
cost is : 0.677910864384884
cost is : 0.6779106325824525
cost is : 0.6779104008934235
cost is : 0.6779101693177412
cost is : 0.6779099378553499
cost is : 0.6779097065061939
cost is : 0.6779094752702178
cost is : 0.6779092441473656
cost is : 0.677909013137582
cost is : 0.6779087822408113
cost is : 0.6779085514569979
cost is : 0.6779083207860879
cost is : 0.6779080902280551
cost is : 0.6779078597828131
cost is : 0.6779076294503068
cost is : 0.6779073992304804
cost is : 0.6779071691232786
cost is : 0.6779069391286463
cost is : 0.6779067092465278
cost is : 0.6779064794768682
cost is : 0.6779062498196119
cost is : 0.6779060202747039
cost is : 0.6779057908420888
cost is : 0.6779055615217114
cost is : 0.6779053323135167
cost is : 0.67790

In [17]:
pathTest = "sample/Test/*"
test_set_orig = np.array([cv2.imread(file) for file in glob.glob(pathTest)])
test_set_flatten = test_set_orig/255
test_set = test_set_flatten.reshape(test_set_flatten.shape[0],test_set_flatten.shape[1]*test_set_flatten.shape[2]*test_set_flatten.shape[3]).T
print(test_set.shape)

results = predict(test_set, parameters)
print(results)

(6075, 8)
[[1. 1. 1. 1. 1. 1. 1. 1.]]
